### Emma Klemets, 260775167   
All my work, code, output and written answers are in this notebook.

In [1]:
#for interactive plots - not what I want

# %matplotlib
# %matplotlib inline
# %matplotlib notebook

In [1]:
import numpy as np
import matplotlib

#for pop out plots - windows
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

Make a 3-D nbody code that calculates the forces by computing the potential, where the potential is found by convolving the density with the (softened) potential from a single particle. The acceleration is then found by taking the gradient of the potential. You will probably wish to use a leapfrog solver with fixed timestep.

In [3]:
"""
calculates the forces by computing the potential

potential is found by convolving the density with the (softened) potential from a single particle. 
phi = rho x V_1

The acceleration is then found by taking the gradient of the potential. 
acc = grad phi


calculate the potential using the Green's function!
So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
Since (taking 4*pi*G=1): laplacian(phi) = rho ,

solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian

- how does it not matter which particle for rho?
- what is the r in G's relative to - should it be between two particles? 
"""
x=1

In [4]:
class Particles_S:
    def __init__(self,x,v,m):
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.f=np.empty(x.shape)
        self.a=np.empty(x.shape)
        self.n=self.x.shape[0]
        
    def r(self):
        return np.sqrt(np.sum(self.x**2, axis=1))
        
    def get_forces(self, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        
    #this is for calculting forces with position values (xx) that are not ever saved
    def get_forces_2(self, xx, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-xx[i,:]+xx[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3

    def get_acc(self, soft=0.01, do_pot=False):
        self.a[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.a[i,:]=self.a[i,:]+dx*r3 #going to be the same really as F as m=1 rn

    
    def take_step_simple(self,dt): #this is bad, but it does what it should - no dt softening here
        self.get_forces()
        self.x=self.x+dt*self.v
        self.v=self.v+dt*self.f
        
    def take_step_better(self, dt, soft=0.01): #works, is decent - also no dt softening
        xx=self.x+0.5*self.v*dt #first half step
        #gets forces for centerpoint of step
        self.get_forces_2(xx, soft)
        f=self.f
        
        #gets velocity for centerpoint of step
        vv=self.v+0.5*dt*f
        self.x=self.x+dt*vv
        self.v=self.v+dt*f

    def take_step_leapfrog(self, dt, soft=0.01):
        self.get_acc()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        

In [6]:
n=2
#for random starting
# x=np.random.randn(n,2)
# v=np.random.randn(n,2)

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=1
x[1,0]=-1
x[0,2]=1
x[1,2]=-1
v[0,1]=0.25
v[1,1]=-0.25

# x[2,0]=-0.5
# v[2,1]=0

m=np.ones(n) #mass
parts=Particles_S(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig, ax = plt.subplots()      

# for k in range(10):
#     for l in range(10):
        
for k in range(100):
    for l in range(20):
#         parts.take_step_simple(dt)
#         parts.take_step_better(dt)
        parts.take_step_leapfrog(dt)
    
    print('step ',k)
#     print(parts.x)
#     f,pot=get_forces(x,do_pot=True)
#     kin=np.sum(v**2)/2
    #print(kin,pot,kin-pot/2)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], color=colors[n_i])
#     plt.axis([-2,2,-2,2])
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)

print()

step  0
step  1
step  2
step  3
step  4
step  5
step  6
step  7
step  8
step  9
step  10
step  11
step  12
step  13
step  14
step  15
step  16
step  17
step  18
step  19
step  20
step  21
step  22
step  23
step  24
step  25
step  26
step  27
step  28
step  29
step  30
step  31
step  32
step  33
step  34
step  35
step  36
step  37
step  38
step  39
step  40
step  41
step  42
step  43
step  44
step  45
step  46
step  47
step  48
step  49
step  50
step  51
step  52


KeyboardInterrupt: 

In [72]:
class Particles:
    def __init__(self,x,v,m, gridSize, grid_dx=1):
        #set up our grid - gridSize should be [x, y, z]
        x_g = np.arange(0, gridSize[0], grid_dx)
        y_g = np.arange(0, gridSize[1], grid_dx)
        z_g = np.arange(0, gridSize[2], grid_dx)
        self.grid = np.meshgrid(x_g, y_g, z_g)
        self.dt = grid_dx
        
        self.x=x.copy()
        self.v=v.copy()
        try:
            self.m=m.copy()
        except:
            self.m=m
        self.f=np.empty(x.shape)
        self.a=np.empty(x.shape)
        self.n=self.x.shape[0]
        
    def r(self):
        return np.sqrt(np.sum(self.x**2, axis=1))
        
    def get_forces(self, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3
        
    #this is for calculting forces with position values (xx) that are not ever saved
    def get_forces_2(self, xx, soft=0.01, do_pot=False):
        self.f[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-xx[i,:]+xx[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    self.f[i,:]=self.f[i,:]+dx*self.m[j]*r3

    def get_acc(self, soft=0.01, do_pot=False):
        self.a[:]=0
        for i in range(self.n):
            for j in range(self.n):
                if i!=j:
                    dx=-self.x[i,:]+self.x[j,:]
                    drsqr=np.sum(dx**2)
                    
                    if drsqr<soft**2:
                        drsqr = soft**2
                
                    r3=1/(drsqr*np.sqrt(drsqr))
                    
                    self.a[i,:]=self.a[i,:]+dx*r3 #going to be the same really as F as m=1 rn


    def take_step_leapfrog(self, dt, soft=0.01):
        self.get_acc()
        v_h = self.v + self.a*dt
        self.x = self.x + v_h*dt
        self.v = v_h #so the V saved will be a half step behind x
        

    """
    acc = grad phi

    calculate the potential using the Green's function!
    So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
    Since (taking 4*pi*G=1): laplacian(phi) = rho ,

    solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian
    """
    
    def r_grid_from_point(self, point):
        t = np.zeros( (len(self.grid[0]), len(self.grid[1]), len(self.grid[2])) ) 

        for i in range(gridSize[0]):
            for j in range(gridSize[0]):
                for k in range(gridSize[0]):
                    r = np.sqrt((self.grid[0][i][j][k] + self.dt/2 - point[0])**2 + \
                                (self.grid[1][i][j][k]+ self.dt/2-point[1])**2 + (self.grid[2][i][j][k]+ self.dt/2-point[2])**2)
                    t[i][j][k] = r
                    
        return t
    
    def G_laplacian(self, soften=0.01):
        #can we just use a non-real particle at the center to make this easier?
        
        centerPoint = [len(self.grid[0])/2.0, len(self.grid[1])/2.0, len(self.grid[2])/2.0]
        R_arr = self.r_grid_from_point(centerPoint)
#         R_arr = np.sqrt(np.sum(self.grid**2, axis=1))

        R_arr[R_arr<soften] = soften
        
        return -1/(4*np.pi*R_arr)
    
    def get_density(self):
        
        #make empty 3D array to hold the total masses and then densities for each cell
#         density = np.meshgrid(self.grid[0][0]*0, self.grid[0][0]*0, self.grid[0][0]*0)
            
        #use a histogram to do way easier
        
        #get our bin edges - need one extra point compared to our grid as we need all the edges
        dt = self.dt
        x_g = np.arange(0, len(self.grid[0])*dt + dt, dt) #these can't deal with particles in -ve space -> BC will deal with this
        y_g = np.arange(0, len(self.grid[1])*dt + dt, dt)
        z_g = np.arange(0, len(self.grid[2])*dt + dt, dt)
        
        #now fill the histgram, with the masses as the weights
        Hist, edges = np.histogramdd(self.x, bins=[x_g,y_g,y_g], weights=self.m)
            
        #get total mass per grid cube, then just divide by the volume of each cube
        density = Hist/dt**3
        return density
     
    def get_acc_conv(self, soft=0.01, do_pot=False):
        #    self.a[:]=0
        #what is rho??  - the single particle potential? 
        # no I think it's the density - and howsome G's is the potential?
        rho = self.get_density()
    
        phi = convFunction(self.G_laplacian(), rho)
        
#         acc = grad phi
        #hmmm how to do grad

#         self.a[i,:] = 0
        return phi
        
        
def convFunction(arr1, arr2, p=5):

    arrFT1 = np.fft.fft(np.pad(arr1, [0, p]))
    arrFT2 = np.fft.fft(np.pad(arr2, [0, p]))
        
    convolved = np.fft.ifft(arrFT1 * arrFT2)/len(arr1)
    
    if p > 0:
        convolved = convolved[:-p,:-p]

    return convolved.real

def green(x, y, soften = 0.01):
    r = rad(x,y)
    r[r<soften] = soften

    g = 1/(2*np.pi) * np.log(r)
    
    return g

In [68]:
gridSize = [5, 5, 5]
grid_dx = 1

x_g = np.arange(0, gridSize[0], grid_dx)
y_g = np.arange(0, gridSize[1], grid_dx)
z_g = np.arange(0, gridSize[2], grid_dx)
grid = np.meshgrid(x_g, y_g, z_g)

t = np.zeros( gridSize)

center = [gridSize[0]/2.0, gridSize[0]/2.0, gridSize[0]/2.0]

for i in range(gridSize[0]):
    for j in range(gridSize[0]):
        for k in range(gridSize[0]):
            r = np.sqrt((grid[0][i][j][k] + dt/2 - center[0])**2 + (grid[1][i][j][k]+ dt/2-center[1])**2 + (grid[2][i][j][k]+ dt/2-center[2])**2)
            t[i][j][k] = r
    
print(t)

# plt.plot(t[0])

SyntaxError: invalid syntax (<ipython-input-68-a90fa360ca87>, line 4)

In [57]:
tryME = grid.copy()
# t = np.meshgrid(grid[0][0]*0, y, z)

grid[0][0][0]

array([0, 0, 0, 0, 0])

In [74]:
n=2
#two particles in circular orbit

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=0.5 #p1 x
x[0,1]=1.5 #p1 y

x[1,0]=2.5 #p2 x
x[1,1]=2.5 #p2 y

v[0,1]=0.25
v[1,1]=-0.25
v[0,2]=0.25
v[1,2]=-0.25

gridSize = [5, 5, 5]

m=np.ones(n) #mass
parts=Particles(x,v,m, gridSize)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# plt.pause(0.01)
# plt.show()

# d = parts.get_density()

# x=np.arange(0, gridSize[0], 1)
# xx,yy=np.meshgrid(x,x)

# for i in range(0, gridSize[0]):
#     plt.pcolor(xx, yy, d[i],  shading='auto')
#     plt.title("level: {}".format(i))
#     plt.pause(2)
# plt.colorbar()

# print(d)


phi_test = parts.get_acc_conv()

print(phi_test)
#well it does somthing rn, but I have a feeling I should have non-zero values like everywhere
#need to get gradient to get actual accs instead of just potential?

"""
colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(300):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    plt.pause(0.01)
"""   

[[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [-5.30516477e-03 -6.49747334e-03 -7.11762543e-03 -6.49747334e-03
   -5.30516477e-03 -4.16333634e-19 -5.55111512e-19 -4.16333634e-19
   -5.55111512e-19 -1.11022302e-18]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
    0.00000000

'\ncolors = cm.rainbow(np.linspace(0, 1, n)) \n\nfig = plt.figure()\nax = Axes3D(fig)\n        \nfor k in range(300):\n    for l in range(20):\n        parts.take_step_leapfrog(dt)\n    \n    print(\'step \',k)\n\n    #     plt.clf() #clears prev graphs\n    for n_i in range(n):\n        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])\n#     ax.set_title("frame {}".format(k))\n#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)\n    plt.pause(0.01)\n'

### Part 1: 
Using this code, show that a single particle starting at rest remains motionless. 

In [ ]:
n=1
#single particle at rest
x=np.random.randn(n,3)
v=np.random.randn(n,3)*0

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n))

fig = plt.figure()
ax = Axes3D(fig)     

for k in range(1000):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
#     print('step ',k)
    
#     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
        
    ax.set_title("frame {}".format(k))
    plt.pause(0.01)


### Part 2: 
Next, show that a pair of particles placed in a circular orbit continue to orbit each other, for at least some reasonable length of time. 

In [ ]:
n=2
#two particles in circular orbit

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=1
x[1,0]=-1
x[0,2]=1
x[1,2]=-1

v[0,1]=0.25
v[1,1]=-0.25
v[0,2]=0.25
v[1,2]=-0.25

m=np.ones(n) #mass
parts=Particles(x,v,m)

soft=0.01
# dt=soft**1.5*0.05
dt=0.01

colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(300):
    for l in range(20):
        parts.take_step_leapfrog(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)
    plt.pause(0.01)

### Part 3: 
Set up both periodic and non-periodic boundary conditions. Set up a problem where hundreds of thousands of particles are initially scattered randomly throughout the domain. Show the evolution with time for both periodic and non-periodic boundary conditions. Track the total energy how well is it conserved? 

### Part 4: 
In cosmology, we start the universe with a scale-invariant power spectrum, so mass fluctuations are proportional to k−3. Start with the particles on a grid, but with masses derived from a realization of k−3 and use periodic boundary conditions (although not strictly necessary, you may with to start with your particles in the center of grid cells rather than at the corners). How does your universe look now?